<a href="https://colab.research.google.com/github/YYJ-SH/aquaponics/blob/main/imgaug%EB%A5%BC_%ED%86%B5%ED%95%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%A6%9D%EA%B0%95_with_%EB%9D%BC%EB%B2%A8_txt%26effects_%EC%A0%80%EC%9E%A5%ED%8F%B4%EB%8D%94%EB%94%B0%EB%A1%9C%EB%94%B0%EB%A1%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import shutil
import imageio

%matplotlib inline

In [3]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [25]:
filepath = "C:\\Users\\user\\Downloads\\lettuce soft rot\\lettuce soft rot" # jpg 사진 파일을 불러올 디렉토리
labelpath = "C:\\Users\\user\\Downloads\\SoftrotLabel" # 라벨 파일이 있는 디렉토리
filepath2 = "C:\\Users\\user\\Downloads\\augedimg" #증강된 사진을 저장할 디렉토리
filepath3 = "C:\\Users\\user\\Downloads\\augedlabel" #증강된 라벨을 저장할 디렉토리
if not os.path.exists(filepath2):
    os.makedirs(filepath2)
    
if not os.path.exists(filepath3):
    os.makedirs(filepath3)

In [16]:
# imgaug는 pascal(절대좌표) 기반이기 때문에, yolo(상대좌표)에서 쓸 수 있게 변환하기 위해 좌표를 바꾸어 주어야 합니다. 
# 좌표를 바꾸어 주는 함수.


def convert(size, box) :
    x1 = (box[1] - box[3] / 2) * size[0]
    x2 = (box[1] + box[3] / 2) * size[0]
    y1 = (box[2] - box[4] / 2) * size[1]
    y2 = (box[2] + box[4] / 2) * size[1]
    return [x1, y1, x2, y2]

In [17]:
# pascal 기반으로 했던 좌표계를 다시 yolo 기반으로 바꾸어 주는 함수입니다


def revert(size, box):  # box : [x1,y1,x2,y2]
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [21]:
#effect2 = "sigcontrast_blur_clahe_blabla"
effect = "sigcontrast_blur_clahe_mtypl_blabla" #사진이름 뒤에 추가할 문자열

def augimg(idx, imgname):

    with open(labelpath + "/" +imgname + ".txt",
              'r') as file:  #labelpath에 라벨이 있는 경로넣기
        lines = file.readlines()

        # Create a list of lists, where each sublist is a line from the file
        lists = [line.strip().split(' ') for line in lines]

        # Convert the strings in each sublist to floats
        lists = [[float(x) for x in sublist] for sublist in lists]
        print("lists::", lists)

        # Set the bound variable to the last sublist in lists

        auglist = []
        for i in lists:
            bound = i
            print(bound[0])
            bound[0] = int(bound[0])# class_num에 0번 원소인 클래스를 저장합니다. 소수로 되어 있기 때문에 다시 정수로 바꿔줍니다.
            print(bound[0])
            class_num = bound[0]
            image = imageio.imread(filepath + "/" + imgname +
                                   ".jpg")  # 이미지를 불러옵니다.
            size = image.shape  # 불러온 이미지의 size를 저장합니다.
            point = convert(
                size, bound
            )  # 상대좌표 (x_start, y_start, width, height) -> 절대좌표 (x_start, y_start, x_end, y_end)

            bbs = BoundingBoxesOnImage([
                BoundingBox(x1=point[0], y1=point[1], x2=point[2], y2=point[3])
            ],
                                       shape=image.shape)

            # 실제 Augmentation 인자를 추가하는 부분. 사진의 효과를 지정해 주는 부분이다. 자세한 내용은 공식문서 참조
            #
            seq = iaa.Sequential(
                [
                    #랜덤으로 선택해서 관점 틀어주는효과
                    # Sometimes(비율, 효과함수)
                    
                    # iaa.imgcorruptlike.Snow(severity=2), # 눈 효과
                    # iaa.Grayscale(alpha=(0.9, 1.0)),  # 회색 효과
                    # iaa.PerspectiveTransform(scale=(0.01, 0.1)),
                    iaa.GaussianBlur(sigma=(0.0, 1.0)),
                    iaa.MultiplyAndAddToBrightness(mul=(0.5, 1.5),add=(-0, 30)),
                   
                    

                    # iaa.Rain(speed=(0.1, 0.3)), #비 효과
                   # iaa.Affine(
                        # scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                       #rotate=(-2, 2),  # rotate by -45 to +45 degrees(회전 효과)
                   # ),
                ],
                random_order=True)

            # 이미지에 대해 Augmentation을 진행합니다. Rotation과 같은 인자가 있다면 Bounding Box의 좌표 역시 바뀌어야 합니다.
            global image_aug
            global bbs_aug
            image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

            before = bbs.bounding_boxes[0]
            after = bbs_aug.bounding_boxes[0]

            aug_box = [after.x1, after.y1, after.x2, after.y2]
            aug_box = revert(size, aug_box)
            aug_box = [class_num] + aug_box

            for i in range(1, len(aug_box)):
                aug_box[i] = round(aug_box[i], 6)
            for i in range(len(aug_box)):
                aug_box[i] = str(aug_box[i])
                auglist.append(aug_box[i])

            
            #with open(f"{filepath2}/{imgname}{idx}"+".txt", 'w') as f :
            """f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2] , ' ', aug_box[3], ' ', aug_box[4], '\n'])
            print("augbox::", [aug_box[0], ' ', aug_box[1], ' ', aug_box[2] , ' ', aug_box[3], ' ', aug_box[4], '\n'])"""

        my_list = auglist
        with open(f"{filepath3}/{imgname}_{idx}_{effect}" + ".txt", 'w') as file:
            # Join the items in the list into a single string, with newline characters as separators
            # and 5 items per line
            text = '\n'.join([
                ' '.join(my_list[i:i + 5]) for i in range(0, len(my_list), 5)
            ])

            # Write the string to the file
            file.write(text)

            # 증강된 이미지를 저장합니다.
        imageio.imwrite(f"{filepath2}/{imgname}_{idx}_{effect}"+".jpg", image_aug)
        print("filename::",f"{filepath2}/{imgname}_{idx}_{effect}"+".jpg")

In [27]:
#텍스트랑 jpg 개수가 다를 때 같은 이름 가진 것만 뽑아내는 함수
import os
import shutil

# directory containing jpg files
jpeg_dir = filepath

# directory containing txt files
txt_dir = labelpath

# directory to move matching txt files to
dest_dir = labelpath+'\\jpgandtxt'
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
    
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
    
    
# get the names of all jpg files in the jpg directory
jpeg_files = [f for f in os.listdir(jpeg_dir) if f.endswith('.jpeg')]

# get the names of all txt files in the txt directory
txt_files = [f for f in os.listdir(txt_dir) if f.endswith('.txt')]

# compare the names of the jpg and txt files
for jpeg_file in jpeg_files:
    for txt_file in txt_files:
        if jpeg_file.split('.')[0] == txt_file.split('.')[0]:
            # move the txt file to the destination directory
            shutil.copy(os.path.join(txt_dir, txt_file), os.path.join(dest_dir, txt_file))
        
        

FileNotFoundError: ignored

In [ ]:
#개수 정확히 맞췄다면 패스
#filepath=dest_dir


In [23]:
# 디렉토리에 있는 txt 파일명을 한꺼번에 리스트에 추가해 주는 코드

root_path = labelpath # 위에서 지정했던 dest_dir이 목적지가 됩니다.

# get a list of all files in the directory
files = os.listdir(root_path)

# create an empty list to store filenames
txt_filenames = []

# iterate over the files
for file in files:
  # check if the file is a JPG file
  if file.endswith('.txt'):
    # remove the '.jpg' from the filename
    filename = file[:-4]
    # add the filename to the list
    txt_filenames.append(filename)

# print the filenames
print(len(txt_filenames))
print(txt_filenames)

FileNotFoundError: ignored

In [ ]:
imgs = txt_filenames

how_many_times =60 #몇 번 증강시킬 것인지?
for i in imgs:
    #try:
        for j in range(
                how_many_times):  # file path내에 있는 각 이미지를 how_many_times장씩 증강합니다.
            augimg(j, i)
            
    #except:
       # pass

lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_0_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_1_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_2_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1


C:\Users\esccy\AppData\Local\Temp\ipykernel_6464\3885331932.py:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filepath + "/" + imgname +


filename:: C:\Users\esccy\Downloads\augedimg/1577601_3_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_4_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_5_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_6_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_7_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_8_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_9_sigcontra

filename:: C:\Users\esccy\Downloads\augedimg/1577601_58_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.392694, 0.48528, 0.594912, 0.792023]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/1577601_59_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_0_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_1_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_2_s

1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_30_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_31_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_32_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0.856771, 0.390625, 0.208333, 0.454861]]
1.0
1
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/5604145_33_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.53125, 0.321181, 0.200521, 0.347222], [1.0, 0.582031, 0.842014, 0.302083, 0.277778], [1.0, 0

filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_4_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_5_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_6_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_7_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_8_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_9_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Download

filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_57_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_58_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.533333, 0.504286, 0.422857, 0.568571]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/gwg-sch2754_59_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_0_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_1_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_2_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
f

filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_46_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_47_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_48_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_49_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_50_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538086, 0.494792, 0.673828]]
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis_51_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.429688, 0.538

1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis1_30_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.324219, 0.5, 0.453125, 0.746094], [1.0, 0.704427, 0.560547, 0.273438, 0.6875]]
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis1_31_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.324219, 0.5, 0.453125, 0.746094], [1.0, 0.704427, 0.560547, 0.273438, 0.6875]]
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis1_32_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.324219, 0.5, 0.453125, 0.746094], [1.0, 0.704427, 0.560547, 0.273438, 0.6875]]
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis1_33_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists:: [[1.0, 0.324219, 0.5, 0.453125, 0.746094], [1.0, 0.704427, 0.560547, 0.273438, 0.6875]]
1.0
1
1.0
1
filename:: C:\Users\esccy\Downloads\augedimg/Y-2006-Unino.Botrytis1_34_sigcontrast_blur_clahe_mtypl_blabla.jpg
lists::

In [ ]:
#아래쪽은 실행시키지 말 것!

import os

# Specify the directory containing the text files
directory = 'C:\\Users\\gbuln\\Desktop\\testReuslts\\documents2'

In [ ]:
# Get a list of all files in the directory
files = os.listdir(directory)

# Iterate over the files
for file in files:
  # Only process .txt files
  if file.endswith('.txt'):
    # Open the file for reading
    with open(os.path.join(directory, file), 'r') as f:
      # Read the file contents
      contents = f.read()
      # Replace 0.0 with 0
      contents = contents.replace('33', '1')
    # Open the file for writing
    with open(os.path.join(directory, file), 'w') as f:
      # Write the modified contents to the file
      f.write(contents)

In [ ]:
import os

# specify the directory where the .txt files are located

# iterate over all files in the directory
for filename in os.listdir(directory):
    # only process .txt files
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                values = line.split(" ")
                if len(values) > 5:
                    # delete the file if there are more than 5 values
                    print(filename)
                    os.remove(file_path)
                    break


In [ ]:
import os

directory = 'C:\\Users\\gbuln\\Desktop\\testReuslts\\documents2'

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path) as f:
            lines = f.readlines()
        with open(file_path, 'w') as f:
            for line in lines:
                elements = line.split(" ")
                elements = elements[:5]
                f.write(" ".join(elements))
